In [1]:
import torch
from torch import nn, Tensor
import numpy as np
from torchvision.utils import save_image

device = "cuda" if torch.cuda.is_available() else "cpu"
device

device(type='cuda')

# 1. Dataset

In [2]:
import torchvision

img_size = 32
    
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((img_size, img_size)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5],
                                     std=[0.5])
])

images = torchvision.datasets.MNIST(root='./mnist_data', train=True, download=True, transform=transform)

In [3]:
BATCH_SIZE = 64
dataloader = torch.utils.data.DataLoader(images, batch_size=BATCH_SIZE, shuffle=True)

# 2. Model

In [4]:
channels = 1
img_shape = (channels, img_size, img_size)
latent_dim = 100

In [5]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(128, 256),
            nn.BatchNorm1d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(256, 512),
            nn.BatchNorm1d(512, 0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
        
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )
        
    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [6]:
class Descriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

In [7]:
generator = Generator()
discriminator = Descriminator()

In [8]:
generator.to(device)

Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=1024, bias=True)
    (12): Tanh()
  )
)

In [9]:
discriminator.to(device)

Descriminator(
  (model): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

# 3. Training

In [10]:
import os
os.makedirs("images", exist_ok=True)

save_interval = 10

In [11]:
EPOCHS = 200

optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

criterion = nn.BCELoss()

hist = {
        "train_G_loss": [],
        "train_D_loss": [],
    }

for epoch in range(EPOCHS):
    running_G_loss = 0.0
    running_D_loss = 0.0

    for i, (imgs, _) in enumerate(dataloader):

        real_imgs = imgs.to(device)
        valid = torch.ones(imgs.shape[0], 1).to(device)
        fake = torch.zeros(imgs.shape[0], 1).to(device)

        # --- Train Generator --- 
        optimizer_G.zero_grad()
        # Noise input for Generator
        z = Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))).to(device)

        gen_imgs = generator(z)
        G_loss = criterion(discriminator(gen_imgs), valid)
        running_G_loss += G_loss.item()

        G_loss.backward()
        optimizer_G.step()

        # --- Train Descriminator --- 
        optimizer_D.zero_grad()
        real_loss = criterion(discriminator(real_imgs), valid)
        fake_loss = criterion(discriminator(gen_imgs.detach()), fake)
        D_loss = (real_loss + fake_loss) / 2
        running_D_loss += D_loss.item()

        D_loss.backward()
        optimizer_D.step()
    
    epoch_G_loss = running_G_loss / len(dataloader)
    epoch_D_loss = running_D_loss / len(dataloader)
    
    print(f"Epoch [{epoch + 1}/{EPOCHS}], Train G Loss: {epoch_G_loss:.4f}, Train D Loss: {epoch_D_loss:.4f}")

    hist["train_G_loss"].append(epoch_G_loss)
    hist["train_D_loss"].append(epoch_D_loss)

    if epoch % save_interval == 0:
        save_image(gen_imgs.data[:25], f"images/epoch_{epoch}.png", nrow=5, normalize=True)


Epoch [1/200], Train G Loss: 1.1855, Train D Loss: 0.4254
Epoch [2/200], Train G Loss: 1.6886, Train D Loss: 0.3632
Epoch [3/200], Train G Loss: 1.7938, Train D Loss: 0.3640
Epoch [4/200], Train G Loss: 1.9682, Train D Loss: 0.3350
Epoch [5/200], Train G Loss: 2.1366, Train D Loss: 0.3066
Epoch [6/200], Train G Loss: 2.1757, Train D Loss: 0.3021
Epoch [7/200], Train G Loss: 2.2030, Train D Loss: 0.3033
Epoch [8/200], Train G Loss: 2.3021, Train D Loss: 0.2936
Epoch [9/200], Train G Loss: 2.4095, Train D Loss: 0.2781
Epoch [10/200], Train G Loss: 2.4214, Train D Loss: 0.2767
Epoch [11/200], Train G Loss: 2.4791, Train D Loss: 0.2683
Epoch [12/200], Train G Loss: 2.4148, Train D Loss: 0.2725
Epoch [13/200], Train G Loss: 2.4165, Train D Loss: 0.2828
Epoch [14/200], Train G Loss: 2.3369, Train D Loss: 0.2827
Epoch [15/200], Train G Loss: 2.2245, Train D Loss: 0.3088
Epoch [16/200], Train G Loss: 2.1508, Train D Loss: 0.3156
Epoch [17/200], Train G Loss: 2.0428, Train D Loss: 0.3360
Epoch 